In [ ]:
import pandas as pd
from collections import Counter
import numpy as np
from matplotlib.table import table
from sklearn.neighbors import KernelDensity
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.gridspec as grid_spec
import jinja2
import re
import tabulate
import ridgeplot as rg
import nbformat
from sympy import true, false

import seaborn as sns
import plotly.io as pio
pio.renderers.default = "notebook"
from IPython.display import IFrame
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle, Paragraph, Spacer
from reportlab.lib import colors
from reportlab.lib.styles import getSampleStyleSheet
#Okay, we have our standardized street names. Now we start seeing how many crashes each has.
df = pd.read_csv("2012-24_Crash_Events_std_street_names_dataset.csv")



In [ ]:
df['Serious_and_Fatal_Injuries'] = df['NumberOfFatalities'] + df['NumberOfSeriousInjuries']

pared_df = []

df['OnStreet_standardized'].value_counts()





In [ ]:
freqtb_y = pd.crosstab(df["Serious_and_Fatal_Injuries"], df["OnStreet_standardized"])
freqtb_y


In [ ]:
TCT10_i = df["OnStreet_standardized"].value_counts().head(10).index
TCT10 = df[df["OnStreet_standardized"].isin(TCT10_i)]
freqtb10_y = pd.crosstab(TCT10["CaseYear"], TCT10["OnStreet_standardized"])

plt.figure(figsize=(10, 6))
sns.heatmap(freqtb10_y.T, annot = True, fmt="d", cmap="magma_r")
plt.title("Serious Injuries & Deaths from the Top 10 Streets By Crashes by Year")
plt.xlabel("Year")
plt.ylabel("Street")
plt.show()

In [ ]:
dropped_df = df[df.OnStreet_standardized != "UNKNOWN"]

TCT10_i = dropped_df["OnStreet_standardized"].value_counts().head(10).index
TCT10 = dropped_df[dropped_df["OnStreet_standardized"].isin(TCT10_i)]
freqtb10_y = pd.crosstab(TCT10["CaseYear"], TCT10["OnStreet_standardized"])

plt.figure(figsize=(10, 6))
sns.heatmap(freqtb10_y.T, annot = True, fmt="d", cmap="magma_r")
plt.title("Serious Injuries/Deaths from the Top 10 Streets By Total Crashes")
plt.xlabel("Year")
plt.ylabel("Street")
plt.show()

In [ ]:
#top streets by number of serious injuries or deaths
streets = dropped_df["OnStreet_standardized"].unique()
years = dropped_df["CaseYear"].unique()
totalseriousinj = []


for street in streets:
    totalseriousinj.append([street, dropped_df[(dropped_df['OnStreet_standardized']==street)].Serious_and_Fatal_Injuries.sum()])
totalseriousinj


In [ ]:
cols = ['Street', 'Serious_and_Fatal_Injuries']
injframe = pd.DataFrame(totalseriousinj, columns=cols)
injframe.sort_values(by='Serious_and_Fatal_Injuries', inplace=True, ascending=False)
injframe



In [ ]:
streets = dropped_df["OnStreet_standardized"].unique()
years = dropped_df["CaseYear"].unique()
totalseriousinj_yearly = []


# Note - not sure how to efficiently select both year and date...
df = df.sort_values('CaseYear')
years.sort()
print(years)
df

In [ ]:
#for year in years:
#    #print(year)
#    year_df = dropped_df.loc[dropped_df['CaseYear'] == year]
#    for street in streets:
#        totalseriousinj_yearly.append([year, street, year_df[(dropped_df['OnStreet_standardized']==street)].Serious_and_Fatal_Injuries.sum()])
#cols = ['CaseYear', 'OnStreet', 'Serious_and_Fatal_Injuries']
#injframe_yearly = pd.DataFrame(totalseriousinj_yearly, columns=cols)
#injframe_yearly.to_csv('yearlyseriousdeaths.csv', index=False)
#Code to generate sums of streets by year. Don't run unless you have to. It takes 20 fucking minutes.

In [ ]:

injframe_yearly = pd.read_csv("yearlyseriousdeaths.csv")

injframe_yearly[injframe_yearly['OnStreet'] == 'STATE ROUTE 104']


In [ ]:
injframe_yearly

for year in years:
    top_2012 = injframe_yearly[injframe_yearly['CaseYear'] == year]
    top_2012=top_2012.sort_values('Serious_and_Fatal_Injuries', ascending=False)
    top = top_2012.head(15)
    name = 'annualcrashlists/'
    name += str(year)
    name += '_top15streetsseriousdeaths.csv'

    top.to_csv(name, index=False)


In [ ]:
freqtb_y = pd.crosstab(top_2012["Serious_and_Fatal_Injuries"], top_2012["OnStreet"])
freqtb_y

TCT10_i = top_2012["OnStreet"].value_counts().head(10).index
TCT10 = top_2012[top_2012["OnStreet"].isin(TCT10_i)]
freqtb10_y = pd.crosstab(TCT10["CaseYear"], TCT10["OnStreet"])

plt.figure(figsize=(10, 6))
sns.heatmap(freqtb10_y.T, annot = True, fmt="d", cmap="magma_r")
plt.title("Serious Injuries/Deaths from the Top 10 Streets By Total Crashes in 2012")
plt.xlabel("Year")
plt.ylabel("Street")
plt.show()